In [1]:
import os
import conda
import pandas as pd
import numpy as np
import json
from scipy.optimize import curve_fit
# from shapely.geometry import Polygon as Poly

pd.options.display.max_columns = 250

conda_file_dir = conda.__file__
conda_dir = conda_file_dir.split('lib')[0]
proj_lib = os.path.join(os.path.join(conda_dir, 'share'), 'proj')
os.environ["PROJ_LIB"] = proj_lib

import matplotlib.pyplot as plt
import matplotlib.cm
import matplotlib.colors as colors

import geopandas
from geopandas.tools import sjoin
import geoplot as gplt
import geoplot.crs as gcrs

import folium
from folium.plugins import MousePosition, Search, FastMarkerCluster

import branca
import branca.colormap as cm

### M3P (ppm) quantiles colormap 

In [2]:
poly  = geopandas.GeoDataFrame.from_file('watershed/huc8sum.shp')
poly['HUC_8'] = poly['HUC_8'].astype(str)
HUC8_json = poly.to_crs(epsg='4326').to_json()

SoilGeoChemical_Pgrouped = pd.read_csv('SoilGeoChemical_Pgrouped.csv', converters={'HUC_8': lambda x: str(x)}).fillna(np.nan)
SoilGeoChemical_Pgrouped_GEOjson = poly.merge(SoilGeoChemical_Pgrouped, on='HUC_8', how = 'left').fillna(np.nan)

m_SoilGeoChemical_P = folium.Map(tiles= 'openstreetmap', location=[48, -102], zoom_start=6)

colormap1 =cm.StepColormap(['#fee5d9','#fcae91','#fb6a4a','#cb181d'], index=SoilGeoChemical_Pgrouped_GEOjson['M3P_ppm'].quantile([0,0.25,0.5,0.75]).to_list(),
                vmin=SoilGeoChemical_Pgrouped_GEOjson['M3P_ppm'].min(),
                vmax=SoilGeoChemical_Pgrouped_GEOjson['M3P_ppm'].max())

SoilGeoChemical_P_dict = SoilGeoChemical_Pgrouped_GEOjson.set_index('HUC_8')['M3P_ppm']
SoilGeoChemical_SoilFertility_dict = SoilGeoChemical_Pgrouped_GEOjson.set_index('HUC_8')['M3P_ppm']
SoilGeoChemical_Ptotal_dict = SoilGeoChemical_Pgrouped_GEOjson.set_index('HUC_8')['p_ppm']

m_SoilGeoChemical_P_layer = folium.GeoJson(
    SoilGeoChemical_Pgrouped_GEOjson.to_json(),
    name='M3P in soil (ppm)',
    style_function=lambda feature: {
        'fillColor': 'grey' if np.isnan(SoilGeoChemical_P_dict[feature['properties']['HUC_8']]) == True 
        else colormap1(SoilGeoChemical_P_dict[feature['properties']['HUC_8']]),
        'fillOpacity': 0.9,
        'color': 'black',
        'weight': 0.9,
    },
    tooltip=folium.features.GeoJsonTooltip(
        fields=['HUC_8', 'M3P_ppm'],
        aliases=['HUC8 watershed', 'M3P in soil (ppm)'], 
        localize=True)
    ).add_to(m_SoilGeoChemical_P)

callback = """\
function (row) {
    var icon, marker;
    icon = L.AwesomeMarkers.icon({
        icon: "map-marker"});
    marker = L.marker(new L.LatLng(row[0], row[1]));
    marker.setIcon(icon);
    return marker;
};
"""
folium.LayerControl().add_to(m_SoilGeoChemical_P)

formatter = "function(num) {returnL.Util.formatNum(num, 3) + ' º ';};"

MousePosition(
    position='bottomright',
    separator=' | ',
    empty_string='NaN',
    lng_first=True,
    num_digits=20,
    prefix='Coordinates:',
    lat_formatter=formatter,
    lng_formatter=formatter,
).add_to(m_SoilGeoChemical_P)

#Customized legend
legend_html =   '''  
                <div id='legend', 
                style="position: fixed; 
                            bottom: 50px; right: 50px; width: 250px; height: 170px; 
                            padding: 10px 10px;
                            border:2px solid grey; z-index:9999; font-size:16px;
                            background:#ffffff;
                            ">
                    <p><strong>M3P in soil (ppm)</strong></p>
                    <p>
                    <span style='color:#fee5d9'>▉</span> 0.20-43.76 (quantile Q1)<br>
                    <span style='color:#fcae91'>▉</span> 43.76-108.31 (quantile Q2)<br>
                    <span style='color:#fb6a4a'>▉</span> 108.31-210.76 (quantile Q3)<br>
                    <span style='color:#cb181d'>▉</span> 210.76-1877.52 (quantile Q4)<br>
                    <span style='color:#848889'>▉</span> No data available<br>
                    </p>
                </div>'
                ''' 
m_SoilGeoChemical_P.get_root().html.add_child(folium.Element(legend_html))


m_SoilGeoChemical_P.save('m_M3P_ppm_quantiles_colormap_.html')

### Soil Fertility colormap

In [30]:
# colormap2=cm.LinearColormap(['yellow','green', 'red'])
# colormap2 = colormap2.to_step(index=[SoilGeoChemical_Pgrouped_GEOjson['M3P_ppm'].min(), 2, 50, 100, SoilGeoChemical_Pgrouped_GEOjson['M3P_ppm'].max()])
colormap2=cm.StepColormap(['yellow','green', 'red'], index=[SoilGeoChemical_Pgrouped_GEOjson['M3P_ppm'].min(), 36, 50,],
                         vmin=SoilGeoChemical_Pgrouped_GEOjson['M3P_ppm'].min(),
                         vmax=SoilGeoChemical_Pgrouped_GEOjson['M3P_ppm'].max())
colormap2

In [13]:
poly  = geopandas.GeoDataFrame.from_file('watershed/huc8sum.shp')
poly['HUC_8'] = poly['HUC_8'].astype(str)
HUC8_json = poly.to_crs(epsg='4326').to_json()

SoilGeoChemical_Pgrouped = pd.read_csv('SoilGeoChemical_Pgrouped.csv', converters={'HUC_8': lambda x: str(x)}).fillna(np.nan)
SoilGeoChemical_Pgrouped_GEOjson = poly.merge(SoilGeoChemical_Pgrouped, on='HUC_8', how = 'left').fillna(np.nan)

m_SoilGeoChemical_P = folium.Map(tiles= 'openstreetmap', location=[48, -102], zoom_start=6)

# colormap1 =cm.StepColormap(['#fee5d9','#fcae91','#fb6a4a','#cb181d'], index=SoilGeoChemical_Pgrouped_GEOjson['M3P_ppm'].quantile([0,0.25,0.5,0.75]).to_list(),
#                 vmin=SoilGeoChemical_Pgrouped_GEOjson['M3P_ppm'].min(),
#                 vmax=SoilGeoChemical_Pgrouped_GEOjson['M3P_ppm'].max())

# colormap2 = cm.LinearColormap(['greenyellow', 'chartreuse', 'limegreen', 'green', 'red'])
# colormap2 = colormap2.to_step(index=[0, 16, 25, 35, 50, SoilGeoChemical_Pgrouped_GEOjson['M3P_ppm'].max()])

colormap2=cm.StepColormap(['yellow','green', 'red'], index=[SoilGeoChemical_Pgrouped_GEOjson['M3P_ppm'].min(), 36, 50,],
                         vmin=SoilGeoChemical_Pgrouped_GEOjson['M3P_ppm'].min(),
                         vmax=SoilGeoChemical_Pgrouped_GEOjson['M3P_ppm'].max())

SoilGeoChemical_P_dict = SoilGeoChemical_Pgrouped_GEOjson.set_index('HUC_8')['M3P_ppm']
SoilGeoChemical_SoilFertility_dict = SoilGeoChemical_Pgrouped_GEOjson.set_index('HUC_8')['M3P_ppm']
SoilGeoChemical_Ptotal_dict = SoilGeoChemical_Pgrouped_GEOjson.set_index('HUC_8')['p_ppm']

m_SoilGeoChemical_SoilFertility_layer = folium.GeoJson(
    SoilGeoChemical_Pgrouped_GEOjson.to_json(),
    name='Soil Fertility Level',
    style_function=lambda feature: {
        'fillColor': 'grey' if np.isnan(SoilGeoChemical_SoilFertility_dict[feature['properties']['HUC_8']]) == True 
        else colormap2(SoilGeoChemical_SoilFertility_dict[feature['properties']['HUC_8']]),
        'fillOpacity': 0.9,
        'color': 'black',
        'weight': 0.9,
    },
#     tooltip=folium.features.GeoJsonTooltip(
#         fields=['HUC_8', 'Soil Fertility Level'],
#         aliases=['HUC8 watershed', 'Soil Fertility Level'], 
#         localize=True)
    ).add_to(m_SoilGeoChemical_P)

callback = """\
function (row) {
    var icon, marker;
    icon = L.AwesomeMarkers.icon({
        icon: "map-marker"});
    marker = L.marker(new L.LatLng(row[0], row[1]));
    marker.setIcon(icon);
    return marker;
};
"""
folium.LayerControl().add_to(m_SoilGeoChemical_P)

formatter = "function(num) {returnL.Util.formatNum(num, 3) + ' º ';};"

MousePosition(
    position='bottomright',
    separator=' | ',
    empty_string='NaN',
    lng_first=True,
    num_digits=20,
    prefix='Coordinates:',
    lat_formatter=formatter,
    lng_formatter=formatter,
).add_to(m_SoilGeoChemical_P)

#Customized legend
legend_html =   '''  
                <div id='legend', 
                style="position: fixed; 
                            bottom: 50px; right: 50px; width: 250px; height: 155px; 
                            padding: 10px 10px;
                            border:2px solid grey; z-index:9999; font-size:16px;
                            background:#ffffff;
                            ">
                    <p><strong>Soil Fertility Level</strong></p>
                    <p>
                    <span style='color:#FFFF00'>▉</span> Deficient (&lt; 36 M3P (ppm))<br>
                    <span style='color:#008000'>▉</span> Optimum (36-50 M3P (ppm))<br>
                    <span style='color:#FF0000'>▉</span> Excessive (&gt; 50 M3P (ppm))<br>
                    <span style='color:#848889'>▉</span> No data available<br>
                    </p>
                </div>'
                ''' 
m_SoilGeoChemical_P.get_root().html.add_child(folium.Element(legend_html))


m_SoilGeoChemical_P.save('m_Soil_Fertility_colormap.html')

### Total P (ppm) quantiles colormap 

In [7]:
SoilGeoChemical_Pgrouped_GEOjson['p_ppm'].quantile([0,0.25,0.5,0.75,1])

0.00      70.000000
0.25     391.666667
0.50     580.000000
0.75     800.000000
1.00    4100.000000
Name: p_ppm, dtype: float64

In [15]:
poly  = geopandas.GeoDataFrame.from_file('watershed/huc8sum.shp')
poly['HUC_8'] = poly['HUC_8'].astype(str)
HUC8_json = poly.to_crs(epsg='4326').to_json()

SoilGeoChemical_Pgrouped = pd.read_csv('SoilGeoChemical_Pgrouped.csv', converters={'HUC_8': lambda x: str(x)}).fillna(np.nan)
SoilGeoChemical_Pgrouped_GEOjson = poly.merge(SoilGeoChemical_Pgrouped, on='HUC_8', how = 'left').fillna(np.nan)

m_SoilGeoChemical_P = folium.Map(tiles= 'openstreetmap', location=[48, -102], zoom_start=6)

colormap3 =cm.StepColormap(['#fee5d9','#fcae91','#fb6a4a','#cb181d'], index=SoilGeoChemical_Pgrouped_GEOjson['p_ppm'].quantile([0,0.25,0.5,0.75]).to_list(),
                vmin=SoilGeoChemical_Pgrouped_GEOjson['p_ppm'].min(),
                vmax=SoilGeoChemical_Pgrouped_GEOjson['p_ppm'].max())

# colormap2=cm.StepColormap(['yellow','green', 'red'], index=[SoilGeoChemical_Pgrouped_GEOjson['M3P_ppm'].min(), 36, 50,],
#                          vmin=SoilGeoChemical_Pgrouped_GEOjson['M3P_ppm'].min(),
#                          vmax=SoilGeoChemical_Pgrouped_GEOjson['M3P_ppm'].max())

SoilGeoChemical_P_dict = SoilGeoChemical_Pgrouped_GEOjson.set_index('HUC_8')['M3P_ppm']
SoilGeoChemical_SoilFertility_dict = SoilGeoChemical_Pgrouped_GEOjson.set_index('HUC_8')['M3P_ppm']
SoilGeoChemical_Ptotal_dict = SoilGeoChemical_Pgrouped_GEOjson.set_index('HUC_8')['p_ppm']

m_SoilGeoChemical_Ptotal_layer = folium.GeoJson(
    SoilGeoChemical_Pgrouped_GEOjson.to_json(),
    name='P in soil (ppm)',
    style_function=lambda feature: {
        'fillColor': 'grey' if np.isnan(SoilGeoChemical_Ptotal_dict[feature['properties']['HUC_8']]) == True 
        else colormap3(SoilGeoChemical_Ptotal_dict[feature['properties']['HUC_8']]),
        'fillOpacity': 0.9,
        'color': 'black',
        'weight': 0.9,
    },
    tooltip=folium.features.GeoJsonTooltip(
        fields=['HUC_8', 'p_ppm'],
        aliases=['HUC8 watershed', 'P in soil (ppm)'], 
        localize=True)
    ).add_to(m_SoilGeoChemical_P)


callback = """\
function (row) {
    var icon, marker;
    icon = L.AwesomeMarkers.icon({
        icon: "map-marker"});
    marker = L.marker(new L.LatLng(row[0], row[1]));
    marker.setIcon(icon);
    return marker;
};
"""
folium.LayerControl().add_to(m_SoilGeoChemical_P)

formatter = "function(num) {returnL.Util.formatNum(num, 3) + ' º ';};"

MousePosition(
    position='bottomright',
    separator=' | ',
    empty_string='NaN',
    lng_first=True,
    num_digits=20,
    prefix='Coordinates:',
    lat_formatter=formatter,
    lng_formatter=formatter,
).add_to(m_SoilGeoChemical_P)

#Customized legend
legend_html =   '''  
                <div id='legend', 
                style="position: fixed; 
                            bottom: 50px; right: 50px; width: 250px; height: 170px; 
                            padding: 10px 10px;
                            border:2px solid grey; z-index:9999; font-size:16px;
                            background:#ffffff;
                            ">
                    <p><strong>Total P in soil (ppm)</strong></p>
                    <p>
                    <span style='color:#fee5d9'>▉</span>70-392 (quantile Q1)<br>
                    <span style='color:#fcae91'>▉</span>392-580 (quantile Q2)<br>
                    <span style='color:#fb6a4a'>▉</span>580-800 (quantile Q3)<br>
                    <span style='color:#cb181d'>▉</span>800-4100 (quantile Q4)<br>
                    <span style='color:#848889'>▉</span> No data available<br>
                    </p>
                </div>'
                ''' 
m_SoilGeoChemical_P.get_root().html.add_child(folium.Element(legend_html))


m_SoilGeoChemical_P.save('m_TotalP_ppm_quantiles_colormap.html')